In [1]:
import os
import open3d
import numpy as np
import pandas as pd

from utils.pcd import *

In [2]:
def inv_transform(T):
    T_inv = np.identity(4)
    T_inv[:3, :3] = T[:3, :3].T
    T_inv[:3, 3] = -np.dot(T[:3, :3].T, T[:3, 3])
    return T_inv

def angles(t):
    rx = math.degrees(math.atan2(t[2][1], t[2][2]))
    ry = math.degrees(math.atan2(-t[2][0], math.sqrt(t[2][1] ** 2 + t[2][2] ** 2)))
    rz = math.degrees(math.atan2(t[1][0], t[0][0]))
    return np.array([rx, ry, rz])

def translation(t):
    return np.array([t[0][3], t[1][3], t[2][3]])

In [15]:
dataset_dir = "D:/Projects/Research/3DMatch/dataset/7-scenes-redkitchen/seq-01/"
fcgf_features_dir = "D:/Projects/Research/3DMatch/features/7-scenes-redkitchen/fcgf"
features_dir = "D:/Projects/Research/3DMatch/features/7-scenes-redkitchen/conv_16_128/"

In [16]:
data = []

for i in range(99):
    src_keypts, src_features, src_scores = read_features_file(f"{fcgf_features_dir}/frame-{i:06d}.npz")
    tgt_keypts, tgt_features, tgt_scores = read_features_file(f"{features_dir}/frame-{(i + 1):06d}.npz", threshold=0.85)
    
    result_ransac = execute_global_registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)
    
    pose1 = np.loadtxt(os.path.join(dataset_dir, f"frame-{1:06d}.pose.txt"))
    pose2 = np.loadtxt(os.path.join(dataset_dir, f"frame-{1 + 1:06d}.pose.txt"))

    gt_tranformation = np.dot(pose1, inv_transform(pose2))

    angle_error = np.mean(np.abs(angles(gt_tranformation) - angles(result_ransac.transformation)))
    translate_error = np.mean(np.abs(translation(gt_tranformation) - translation(result_ransac.transformation)))
    
    # print(f"{i:02d} & {i+1:02d} - Error: {angle_error}, {translate_error}")
    # if angle_error > 10:
    #     src_keypts.paint_uniform_color([1, 0.706, 0])
    #     tgt_keypts.paint_uniform_color([0, 0.651, 0.929])

    #     src_keypts.transform(result_ransac.transformation)

    #     open3d.visualization.draw_geometries([src_keypts, tgt_keypts])
    
    data.append([i, i+1, len(src_keypts.points), len(tgt_keypts.points), len(result_ransac.correspondence_set), angle_error, translate_error])
    
df = pd.DataFrame(data, columns=["source", "target", "src_pts", "tgt_pts", "num_matches", "angle_error", "translation_error"])

In [17]:
threshold = 5
df.loc[:, "correct"] = df.loc[:, "angle_error"].apply(lambda x: 0 if x > threshold else 1)

accuracy = df.correct.sum() / df.shape[0] * 100
angle_error_mean = df[df.correct == 1]["angle_error"].mean()
translate_error_mean = df[df.correct == 1]["translation_error"].mean()

print(f"Accuracy            : {accuracy}")
print(f"Angle error         : {angle_error_mean}")
print(f"Translation error   : {translate_error_mean}")

In [38]:
i = 1

src_keypts, src_features, src_scores = read_features_file(f"{fcgf_features_dir}/frame-{i:06d}.npz")
tgt_keypts, tgt_features, tgt_scores = read_features_file(f"{features_dir}/frame-{(i + 1):06d}.npz")

result_ransac = execute_global_registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)
to_print = f"Keypts: [{len(src_keypts.points)}, {len(tgt_keypts.points)}]\t"
to_print += f"No of matches: {len(result_ransac.correspondence_set)}"

print(to_print)

src_keypts.paint_uniform_color([1, 0.706, 0])
tgt_keypts.paint_uniform_color([0, 0.651, 0.929])

src_keypts.transform(result_ransac.transformation)

open3d.visualization.draw_geometries([src_keypts, tgt_keypts])

In [3]:
dataset_dir = "D:/Projects/Research/3DMatch/dataset/7-scenes-redkitchen/seq-01/"
features_dir = "D:/Projects/Research/3DMatch/estimated_depth/7-scenes-redkitchen/seq-01"

In [9]:
data = []

for i in range(10):
    src_keypts, src_features, src_scores = read_features_file(f"{features_dir}/frame-{i:06d}.estimated.npz")
    tgt_keypts, tgt_features, tgt_scores = read_features_file(f"{features_dir}/frame-{(i + 5):06d}.original.npz")
    
    result_ransac = execute_global_registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)
    
    pose1 = np.loadtxt(os.path.join(dataset_dir, f"frame-{1:06d}.pose.txt"))
    pose2 = np.loadtxt(os.path.join(dataset_dir, f"frame-{1 + 5:06d}.pose.txt"))

    gt_tranformation = np.dot(pose1, inv_transform(pose2))

    angle_error = np.mean(np.abs(angles(gt_tranformation) - angles(result_ransac.transformation)))
    translate_error = np.mean(np.abs(translation(gt_tranformation) - translation(result_ransac.transformation)))
    
    data.append([i, i+1, len(src_keypts.points), len(tgt_keypts.points), len(result_ransac.correspondence_set), angle_error, translate_error])
    
df = pd.DataFrame(data, columns=["source", "target", "src_pts", "tgt_pts", "num_matches", "angle_error", "translation_error"])

In [10]:
threshold = 5
df.loc[:, "correct"] = df.loc[:, "angle_error"].apply(lambda x: 0 if x > threshold else 1)

accuracy = df.correct.sum() / df.shape[0] * 100
angle_error_mean = df[df.correct == 1]["angle_error"].mean()
translate_error_mean = df[df.correct == 1]["translation_error"].mean()

print(f"Accuracy            : {accuracy}")
print(f"Angle error         : {angle_error_mean}")
print(f"Translation error   : {translate_error_mean}")

Accuracy            : 100.0
Angle error         : 2.762354749646699
Translation error   : 0.09595174847227764


In [11]:
src_keypts.paint_uniform_color([1, 0.706, 0])
tgt_keypts.paint_uniform_color([0, 0.651, 0.929])

src_keypts.transform(result_ransac.transformation)

open3d.visualization.draw_geometries([src_keypts, tgt_keypts])

In [14]:
i = 20
src_keypts, src_features, src_scores = read_features_file(f"{features_dir}/frame-{i:06d}.estimated.npz")
tgt_keypts, tgt_features, tgt_scores = read_features_file(f"{features_dir}/frame-{(i + 25):06d}.original.npz")

result_ransac = execute_global_registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)

In [15]:
to_print = f"Keypts: [{len(src_keypts.points)}, {len(tgt_keypts.points)}]\t"
to_print += f"No of matches: {len(result_ransac.correspondence_set)}"

print(to_print)

Keypts: [21314, 8824]	No of matches: 5246


In [16]:
src_keypts.paint_uniform_color([1, 0.706, 0])
tgt_keypts.paint_uniform_color([0, 0.651, 0.929])

src_keypts.transform(result_ransac.transformation)

open3d.visualization.draw_geometries([src_keypts, tgt_keypts])

In [18]:
pose1 = np.loadtxt(os.path.join(dataset_dir, f"frame-{1:06d}.pose.txt"))
pose2 = np.loadtxt(os.path.join(dataset_dir, f"frame-{1 + 25:06d}.pose.txt"))

gt_tranformation = np.dot(pose1, inv_transform(pose2))

angle_error = np.sum(np.abs(angles(gt_tranformation) - angles(result_ransac.transformation)))
translate_error = np.sum(np.abs(translation(gt_tranformation) - translation(result_ransac.transformation)))

In [20]:
angle_error, translate_error

(10.438256083077484, 0.20384485255769003)